In [1]:
from google.colab import drive
drive.mount('/content/drive')\

Mounted at /content/drive


In [2]:
from google.colab import output
# !cp 파일1 파일2 # 파일1을 파일2로 복사 붙여넣기
!cp -r "/content/drive/MyDrive/21-1/data" "data"

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import imutils
import zipfile
import os
from PIL import Image
from typing import Tuple, Sequence, Callable
import glob
from sklearn.model_selection import KFold
import time
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
from google.colab import output

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # 디바이스 설정

In [4]:
os.chdir('/content/data')

from google.colab import output
# 현재 디렉터리에 dirty_mnist라는 폴더 생성
!mkdir "./dirty_mnist_2nd"
#dirty_mnist.zip라는 zip파일을 dirty_mnist라는 폴더에 압축 풀기
!unzip "dirty_mnist_2nd.zip" -d "./dirty_mnist_2nd/"
# 현재 디렉터리에 test_dirty_mnist라는 폴더 생성
!mkdir "./test_dirty_mnist_2nd"
#test_dirty_mnist.zip라는 zip파일을 test_dirty_mnist라는 폴더에 압축 풀기
!unzip "test_dirty_mnist_2nd.zip" -d "./test_dirty_mnist_2nd/"
# 출력 결과 지우기
output.clear()

In [46]:
os.chdir('/content/data')
labels_df = pd.read_csv('dirty_mnist_2nd_answer.csv')[:]
imgs_dir = np.array(sorted(glob.glob('/content/data/dirty_mnist_2nd/*')))[:]
labels = np.array(labels_df.values[:,1:])
test_imgs_dir = np.array(sorted(glob.glob('/content/data/test_dirty_mnist_2nd/*')))

In [6]:
labels_df

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,0,1,1,0,1,0,1,0,0,0,0,1,1,0,0,1,1,0,1,1,0,1,0,0,1,1,1
1,1,1,0,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,0,1,1
2,2,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,1,1,1,0,1,1,1,0
3,3,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,1,1,0,1,1,0,1,1,0,1,0
4,4,0,1,0,1,0,1,0,1,1,0,1,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,0,1,1,0,0,0,0,1,0,0,1,1,0,1,1,0,0,0,0,1,0,0,0,1,1,0
49996,49996,0,1,0,1,0,1,1,1,0,1,0,0,0,0,1,0,0,1,1,1,0,1,0,0,0,1
49997,49997,0,1,0,0,1,1,1,1,0,0,1,1,0,0,0,0,1,0,0,0,0,1,1,1,0,0
49998,49998,0,1,1,1,0,0,1,1,0,1,1,1,1,1,0,0,0,1,1,1,0,0,0,1,0,0


In [7]:
imgs_dir

array(['/content/data/dirty_mnist_2nd/00000.png',
       '/content/data/dirty_mnist_2nd/00001.png',
       '/content/data/dirty_mnist_2nd/00002.png', ...,
       '/content/data/dirty_mnist_2nd/49997.png',
       '/content/data/dirty_mnist_2nd/49998.png',
       '/content/data/dirty_mnist_2nd/49999.png'], dtype='<U39')

In [8]:
labels.shape

(50000, 26)

In [9]:
class MnistDataset_v1(Dataset):
    def __init__(self, imgs_dir=None, labels=None, transform=None, train=True):
        self.imgs_dir = imgs_dir
        self.labels = labels
        self.transform = transform
        self.train = train
        pass
    
    def __len__(self):
        # 데이터 총 샘플 수
        return len(self.imgs_dir)
    
    def __getitem__(self, idx):
        # 1개 샘플 get
        img = cv2.imread(self.imgs_dir[idx], cv2.IMREAD_COLOR)
        img = self.transform(img)
        if self.train==True:
            label = self.labels[idx]
            return img, label
        else:
            return img
        
        pass

In [10]:
#shuffle = torch.hub.load('pytorch/vision:v0.6.0', 'shufflenet_v2_x1_0', pretrained=True)
shuffle = torch.hub.load('pytorch/vision:v0.6.0', 'shufflenet_v2_x0_5', pretrained=True)

class my_Shuffle(nn.Module):
    def __init__(self):
        super(my_Shuffle, self).__init__()
        self.shuffle = shuffle 
        self.FC = nn.Linear(1000, 26)
        nn.init.xavier_normal_(self.FC.weight)
      
    def forward(self, x):
        x = self.shuffle(x)
        x = torch.sigmoid(self.FC(x))
        return x

model = my_Shuffle()
from torchsummary import summary
model.to(device)
summary(model,(3,256,256))

Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /root/.cache/torch/hub/v0.6.0.zip
Downloading: "https://download.pytorch.org/models/shufflenetv2_x0.5-f707e7126e.pth" to /root/.cache/torch/hub/checkpoints/shufflenetv2_x0.5-f707e7126e.pth



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 24, 128, 128]             648
       BatchNorm2d-2         [-1, 24, 128, 128]              48
              ReLU-3         [-1, 24, 128, 128]               0
         MaxPool2d-4           [-1, 24, 64, 64]               0
            Conv2d-5           [-1, 24, 32, 32]             216
       BatchNorm2d-6           [-1, 24, 32, 32]              48
            Conv2d-7           [-1, 24, 32, 32]             576
       BatchNorm2d-8           [-1, 24, 32, 32]              48
              ReLU-9           [-1, 24, 32, 32]               0
           Conv2d-10           [-1, 24, 64, 64]             576
      BatchNorm2d-11           [-1, 24, 64, 64]              48
             ReLU-12           [-1, 24, 64, 64]               0
           Conv2d-13           [-1, 24, 32, 32]             216
      BatchNorm2d-14           [-1, 24

In [11]:
kf = KFold(n_splits=5, shuffle=True)
folds=[]
for train_idx, valid_idx in kf.split(labels_df):
    folds.append((train_idx, valid_idx))

In [16]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True

In [17]:
#seed_everything(42)
for fold in range(1):
    model = my_Shuffle().to(device)
#     model = nn.DataParallel(model)
    train_idx = folds[fold][0]
    valid_idx = folds[fold][1]

    train_transform = transforms.Compose([                                
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
        ])
    valid_transform = transforms.Compose([                                 
        transforms.ToTensor(),
        transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
        ])


    epochs=20
    batch_size=32   # 자신의 VRAM에 맞게 조절해야 OOM을 피할 수 있습니다.
    
    
    # Data Loader
    train_dataset = MnistDataset_v1(imgs_dir=imgs_dir[train_idx], labels=labels[train_idx], transform=train_transform)
    train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

    valid_dataset = MnistDataset_v1(imgs_dir=imgs_dir[valid_idx], labels=labels[valid_idx], transform=valid_transform)
    valid_loader = DataLoader(dataset=valid_dataset, batch_size=32, shuffle=False)  
    
    
    # optimizer
    # polynomial optimizer를 사용합니다.
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones= [23,29], gamma=0.1)

    criterion = torch.nn.BCELoss()
    
    
    epoch_accuracy = []
    valid_accuracy = []
    valid_losses=[]
    valid_best_accuracy=0

    best_models=[]

    for epoch in range(epochs):
      with tqdm(train_loader,total=train_loader.__len__(),unit='batch') as train_bar:
        model.train()
        batch_accuracy_list = []
        batch_loss_list = []
        start=time.time()
        for n, (X, y) in enumerate((train_bar)):
            train_bar.set_description(f"Train Epoch {epoch}")
            X = torch.tensor(X, device=device, dtype=torch.float32)
            y = torch.tensor(y, device=device, dtype=torch.float32)
            y_hat = model(X)
            
            
            optimizer.zero_grad()
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()


            
            y_hat  = y_hat.cpu().detach().numpy()
            y_hat = y_hat>0.5
            y = y.cpu().detach().numpy()

            batch_accuracy = (y_hat == y).mean()
            batch_accuracy_list.append(batch_accuracy)
            batch_loss_list.append(loss.item())
            train_acc = np.mean(batch_accuracy_list)
            
            train_bar.set_postfix(train_loss= loss.item(),train_acc = train_acc)

        model.eval()
        valid_batch_accuracy=[]
        valid_batch_loss = []

        with torch.no_grad():
          with tqdm(valid_loader,total=valid_loader.__len__(),unit="batch") as valid_bar:
            for n_valid, (X_valid, y_valid) in enumerate((valid_bar)):
                valid_bar.set_description(f"Valid Epoch {epoch}")
                X_valid = torch.tensor(X_valid, device=device)#, dtype=torch.float32)
                y_valid = torch.tensor(y_valid, device=device, dtype=torch.float32)
                y_valid_hat = model(X_valid)
                
                valid_loss = criterion(y_valid_hat, y_valid).item()
                
                y_valid_hat = y_valid_hat.cpu().detach().numpy()>0.5
                
                
                valid_batch_loss.append(valid_loss)
                valid_batch_accuracy.append((y_valid_hat == y_valid.cpu().detach().numpy()).mean())
                val_acc=np.mean(valid_batch_accuracy)
                valid_bar.set_postfix(valid_loss = valid_loss,valid_acc = val_acc)
                
            valid_losses.append(np.mean(valid_batch_loss))
            valid_accuracy.append(np.mean(valid_batch_accuracy))

        scheduler.step()

        if np.mean(valid_batch_accuracy) > 0.78:
            path = "/content/drive/MyDrive/21-1/models_3/"
            MODEL = "BEST_ShuffleNetV2"
            torch.save(model, f'{path}_{MODEL}_{valid_loss:2.4f}_epoch_{epoch}.pth')

        if np.mean(valid_batch_accuracy)>valid_best_accuracy:
            best_model=model
            valid_best_accuracy = np.mean(valid_batch_accuracy)
            path = "/content/drive/MyDrive/21-1/models_3/"
            MODEL = "0222_ShuffleNetV2"
            torch.save(model, f'{path}_{MODEL}_{valid_loss:2.4f}_epoch_{epoch}.pth')


    best_models.append(best_model)

Train Epoch 0:   0%|          | 0/625 [00:00<?, ?batch/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
Valid Epoch 0:   0%|          | 0/313 [00:00<?, ?batch/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:94: UserWarning: To copy construct from a 

In [ ]:
best_models

In [52]:
test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225])
        ])
test_dataset = MnistDataset_v1(imgs_dir=test_imgs_dir, transform=test_transform, train=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [54]:
submission = pd.read_csv("sample_submission.csv")

for model in best_models:
    with torch.no_grad():
        model.eval()

        test_dataset = MnistDataset_v1(imgs_dir=test_imgs_dir, transform=test_transform, train=False)
        test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

        for n, X_test in enumerate(tqdm(test_loader)):
            X_test = torch.tensor(X_test, device=device, dtype=torch.float32)
            with torch.no_grad():
                model.eval()  
                pred_test = model(X_test).cpu().detach().numpy()
                submission.iloc[n*32:(n+1)*32,1:] += pred_test






  0%|          | 0/157 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()





  1%|▏         | 2/157 [00:00<00:14, 10.62it/s]




  3%|▎         | 4/157 [00:00<00:13, 11.04it/s]




  4%|▍         | 6/157 [00:00<00:13, 11.41it/s]




  5%|▌         | 8/157 [00:00<00:12, 11.68it/s]




  6%|▋         | 10/157 [00:00<00:12, 11.87it/s]




  8%|▊         | 12/157 [00:00<00:12, 12.07it/s]




  9%|▉         | 14/157 [00:01<00:11, 12.12it/s]




 10%|█         | 16/157 [00:01<00:11, 12.12it/s]




 11%|█▏        | 18/157 [00:01<00:11, 12.25it/s]




 13%|█▎        | 20/157 [00:01<00:11, 12.33it/s]




 14%|█▍        | 22/157 [00:01<00:10, 12.33it/s]




 15%|█▌        | 24/157 [00:01<00:10, 12.45it/s]






In [55]:
submission.iloc[:,1:] = np.where(submission.values[:,1:]>=0.5, 1,0)
submission

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,50000,0,1,1,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0
1,50001,0,1,1,1,0,0,1,0,1,0,0,1,0,0,0,1,1,1,1,0,0,1,0,0,0,0
2,50002,0,0,1,1,1,1,1,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0
3,50003,1,1,0,0,0,0,1,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0
4,50004,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,54995,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0
4996,54996,1,0,1,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
4997,54997,0,0,1,1,0,1,0,0,0,1,1,0,1,0,1,0,1,0,1,1,1,0,0,0,1,0
4998,54998,0,0,1,1,0,0,1,0,1,1,0,1,0,0,0,1,1,1,1,0,0,0,0,0,0,0


In [56]:
submission.to_csv('PREDICTION;ShuffleNetV2_prediction.csv', index=False)

In [59]:
pred=pd.read_csv('/content/data/PREDICTION;ShuffleNetV2_prediction.csv')
pred.head()

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,50000,0,1,1,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0
1,50001,0,1,1,1,0,0,1,0,1,0,0,1,0,0,0,1,1,1,1,0,0,1,0,0,0,0
2,50002,0,0,1,1,1,1,1,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0
3,50003,1,1,0,0,0,0,1,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0
4,50004,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,1,0,0,0
